# LLMs and RAG with DataChain

In LLM applications nowadays, the emerging standard pattern for most use-cases is to employ a pre-trained model with an API from a 3rd party provider and to augment it with a RAG context. On one hand, this means there is not much actual machine learning going on on the user's end. On the other hand naive application of "latest and greatest" models with no prompt engineering, testing and evaluation of RAG context can lead to needlessly expensive operational costs at best and dissapointingly poor performance at worst.

Therefore, even if there is no machine learning involved, there is still a lot of fine tuning we need to do and a lot of that involves large datasets (such as histories of chatbot conversations or large collections of company documents). Just like with ML training, we need to version all that data as we finetune our applications to be able to correctly evaluate the effect of any changes we apply to our models. We can experiment with the LLM choice, prompt engineering, the way we process data for our RAG context (pre-processing, embedding, ...) and so on.

In this example, we will see how we can use DataChain to create such a controlled development environment and how it can help us when we evaluate any fine-tuning of our LLM applications.

We will see how to use DataChain to version our RAG context datasets to preserve reproducibility of our fine-tuning experiments as the RAG context changes. We will also see how to use DataChain in the evaluation of fine-tuning by comparing two different text embedding models and saving (and versioning) the results with additional context.

## Processing a large collection of documents

Let's say that we have a collection of relevant documents which we want to use as context in LLM queries in our chatbot application. We will be using DataChain to create, store and version vector embeddings of our documents.

In this example we will be using papers from the [Neural Information Processing Systems](https://papers.neurips.cc/paper/) conference. 

We will proceed in the following steps:
1. [Data ingestion with DataChain](#data-ingestion) - we will use DataChain to ingest the data, taking advantage of its lazy evaluation feature to only ingest the data we need
1. [Data processing with the Unstructured Python library](#processing-the-documents-individually)
1. [Scaling the data processing with DataChain](#processing-the-documents-at-scale-using-datachain-udfs)
1. [Using Datachain to evaluate different embedding models](#evaluation)
1. [Adding extra context by combining datasets](#adding-more-context---merging-datasets)

In [1]:
from typing import List, Dict, Optional
from collections.abc import Iterator

from datachain.lib.dc import DataChain, C
from datachain.lib.data_model import DataModel
from datachain.lib.file import File
from datachain.sql.functions.array import cosine_distance, euclidean_distance

from unstructured.partition.pdf import partition_pdf
from unstructured.chunking.title import chunk_by_title

from unstructured.embed.huggingface import HuggingFaceEmbeddingConfig, HuggingFaceEmbeddingEncoder

### Data ingestion

We will first ingest the dataset. The data are saved on a cloud storage, so we use the `.from_storage` DataChain method. We will also use the `.filter` method to restrict ourselves only to `.pdf` files (the storage contains many other data which we do not need).

Notice that:

1. Since DataChain employs lazy evaluation, no data are actually loaded just yet (until we invoke an action such as showing or saving our DataChain)
1. The previous point also means that when we filter out all non-pdf files, DataChain doesn't actually waste time loading their content only to throw them away later. This makes DataChain a lot more scalable than tools with eager evaluation.
1. The `.from_storage` method of DataChain operates on the level of the entire bucket. This means that even if the files are stored using a complicated directory structure and potentially uploaded irregularly into this structure, we can retrieve or update our DataChain of articles with just a simple one-line command

In [2]:
dc_papers = (
    DataChain.from_storage("gs://datachain-demo/neurips")
    .filter(C.name.glob("*.pdf"))
    )

In [3]:
dc_papers.show(3)

Listing gs://datachain-demo: 51067 objects [00:12, 3956.71 objects/s]
Processed: 738 rows [00:00, 13740.71 rows/s]


,source,parent,name,version,etag,size,vtype,location,file.source,file.parent,file.name,file.size,file.version,file.etag,file.is_latest,file.last_modified,file.location,file.vtype
0,gs://datachain-demo,neurips/1987/file,02e74f10e0327ad868d138f2b4fdd6f0-Paper.pdf,1721047139405563,CPudi5uIqYcDEAE=,2291566,,None,gs://datachain-demo,neurips/1987/file,02e74f10e0327ad868d138f2b4fdd6f0-Paper.pdf,2291566,1721047139405563,CPudi5uIqYcDEAE=,1,1970-01-01 00:00:00+00:00,None,
1,gs://datachain-demo,neurips/1987/file,03afdbd66e7929b125f8597834fa83a4-Paper.pdf,1721047138865046,CJaf6pqIqYcDEAE=,1322648,,None,gs://datachain-demo,neurips/1987/file,03afdbd66e7929b125f8597834fa83a4-Paper.pdf,1322648,1721047138865046,CJaf6pqIqYcDEAE=,1,1970-01-01 00:00:00+00:00,None,
2,gs://datachain-demo,neurips/1987/file,072b030ba126b2f4b2374f342be9ed44-Paper.pdf,1721046993295769,CJmztdWHqYcDEAE=,1220711,,None,gs://datachain-demo,neurips/1987/file,072b030ba126b2f4b2374f342be9ed44-Paper.pdf,1220711,1721046993295769,CJmztdWHqYcDEAE=,1,1970-01-01 00:00:00+00:00,None,


[limited by 3 objects]


DataChain created a record for each `pdf` file in the `neurips` directory, generating a `file` signal for each file. The file signal contains subsignals with metadata about each file, like `file.name` and `file.size`. Aggregate signals like `file` that contain multiple subsignals are called features.

You can use the `file` feature to not only get metadata about each file, but also to open and read the file as needed.

### Processing the documents individually

We now want to ingest the content of the pdf files as text, divide it into chunks and vectorize those for our RAG application. We are interested in comparing two different models for embeddings. Normally, we would also do some pre-processing and cleaning of the text before vectorization, but we will skip it here for brevity.

We will first do all this with an example of a single pdf using the `unstructured` Python library and then we will see how we can scale this up to the entire bucket with the help of DataChain.

First, we ingest and partition the pdf file and chunk it.


In [4]:
chunks = chunk_by_title(partition_pdf(filename="sample.pdf"))

Next, we vectorize each chunk using HuggingFace embedding encoders

In [6]:
# Define embedding encoder

embedding_encoder = HuggingFaceEmbeddingEncoder(
     config=HuggingFaceEmbeddingConfig()
)

embedding_encoder_alt = HuggingFaceEmbeddingEncoder(
     config=HuggingFaceEmbeddingConfig(model_name='intfloat/e5-small-v2')
)

chunks_embedded = embedding_encoder.embed_documents(chunks)
chunks_embedded_alt = embedding_encoder_alt.embed_documents(chunks)

/home/tibor/Repos/datachain/.venv/lib64/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


We now have our chunks vectorized and ready for comparison (e.g. with cosine similarity). However, we are missing a few ingredients:

1. ***Scaling*** - we only processed a single pdf file and we had to manually specify its path. We need to find a way to process all our documents at scale instead and to save the results.
2. ***Saving and Versioning*** - even if we only had a single or a few PDF files we would like to use in our RAG, it is a good practice to version the outputs so that we can keep track of and fine-tune our RAG application. If we simply save the current results to a bucket and overwrite it each time the source is updated, we lose this. We could version the results manually, e.g. by adding a timestamp to the blob name, but that is not very reliable and will lead to unnecessary copies of files.

### Processing the documents at scale, using DataChain UDFs

We will now use DataChain to solve the scaling and versioning issues we outline above. We will create a DataChain user-defined function (UDF) to process all our PDF files the way we did above with a single file (without us having to manually provide file paths) and save the outputs in a Datachain.

The DataChain UDF functionality will allow us to generate additonal columns in our DataChain, iterating over each of the files listed in it.

We first need to define a DataModel class, which will define the types of our outputs. Inputs and outputs need to be specified like this when we use custom functinos in Datachain.

In [ ]:
# Define the output as a Feature class
class Chunk(DataModel):
    key: str
    text: str
    embeddings: List[float]
    embeddings_alt: List[float]

In the above we define `Chunk` by specifying the names and types of new columns on the output.

We then define our processing function `pdf_chuks`:

In [7]:
# Use signatures to define input/output types (these can be Feature or regular Python types)
def pdf_chunks(file: File) -> Iterator[Chunk]:
    # Ingest the file
    with file.open() as f:
        chunks = chunk_by_title(partition_pdf(file=f))

    chunks_embedded = embedding_encoder.embed_documents(chunks)
    chunks_embedded_alt = embedding_encoder_alt.embed_documents(chunks)

    # Add new rows to DataChain
    for chunk, chunk_alt in zip(chunks_embedded, chunks_embedded_alt):
        record = {}
        record["key"] = file.name.removesuffix("-Paper.pdf")
        record["text"] = chunk.text
        record["embeddings"] = chunk.embeddings
        record["embeddings_alt"] = chunk_alt.embeddings

        yield Chunk(**record)

Here, the syntax is the same as with any other Python function, except that we specify the input and output types using type hints

```
def pdf_chunks(file: File) -> Iterator[Chunk]:
```
Here, `file` specifies that we pass all `file` columns of the original dataset on the input and `Iterator[Chunk]` specifies that we get a bunch of `Chunk` rows on the output (from a single row of the original datachain representing a single paper we will get a new dataset with multiple rows per paper, each representing a single chunk).

We then specify what each row should contain by defining the `record` dictionary and then we use `yield Chunk(**record)` to create the new rows for each input row.

In [8]:
dc_chunks_embeddings = (
    dc_papers
    .limit(20) # we limit ourselves to 20 papers here, to speed up the demo
    .gen(document=pdf_chunks)
)

dc_chunks_embeddings.save()

Processed: 738 rows [00:00, 10743.88 rows/s]
Processed: 0 rows [00:00, ? rows/s]
Processed: 2 rows [00:06,  3.45s/ rows]
Processed: 3 rows [00:12,  4.26s/ rows]
Processed: 4 rows [00:17,  4.60s/ rows]
Processed: 5 rows [00:24,  5.38s/ rows]
Processed: 6 rows [00:29,  5.36s/ rows]
Processed: 7 rows [00:34,  5.16s/ rows]
Processed: 8 rows [00:40,  5.44s/ rows]
Processed: 9 rows [00:47,  5.87s/ rows]
Processed: 10 rows [00:53,  6.01s/ rows]
Processed: 11 rows [01:03,  7.19s/ rows]
Processed: 12 rows [01:10,  7.05s/ rows]
Processed: 13 rows [01:15,  6.56s/ rows]
Processed: 14 rows [01:21,  6.42s/ rows]
Processed: 15 rows [01:26,  5.92s/ rows]
Processed: 16 rows [01:33,  6.14s/ rows]
Processed: 17 rows [01:38,  5.83s/ rows]
Processed: 18 rows [01:42,  5.38s/ rows]
Processed: 19 rows [01:49,  5.99s/ rows]
Processed: 20 rows [01:56,  6.25s/ rows]
Download: 51.2MB [02:00, 444kB/s]
Processed: 20 rows [02:02,  6.11s/ rows]
Generated: 1346 rows [01:55, 11.68 rows/s]


In the cell above we apply our new `pdf_chunks` function to the DataChain `dc_papers`. We do that by using the `gen` method of DataChain with `pdf_chunks`as its parameter. 

`DataChain.gen` is used when we have a function that creates multiple rows per single row of the original datachain (like in our examples, where each paper is split into multiple chunks)

We also presisted the result by the `.save` method.

### Evaluation

We will now use DataChain to calculate the similarity between the two alternative embeddings of each chunk and for further evaluation we will save dataset containing the chunks that differ the most between the two embeddings.



In [ ]:
# Calculate similarity

(
    dc_chunks_embeddings
    .select("document.text", "document.embeddings", "document.embeddings_alt", "document.key")
    .mutate(
        cos_dist=cosine_distance(C("document.embeddings"), C("document.embeddings_alt")),
        eucl_dist=euclidean_distance(C("document.embeddings"), C("document.embeddings_alt")),
    )
    .filter(C("cos_dist") > 0.1)
    .save("embeddings-differences")
)

In the above, we use the `cosine_distance` and `euclidean_distance` Datachain built-in functions to calculate the similarity between the two embeddings for each chunk. To specify that we want to compare columns we use the `C` class from `datachain.lib.dc`.

We use the `mutate` method of datachain, which is a way to add new columns to an existing dataset.

Finally we use the `.save` method once again, but this time with a specific name. This will permanently save and version the datachain as a dataset with the name `embeddings-differences`. Whenever we call `.save("embeddings-differences")` again, a new version of this dataset will be saved automatically, so we can recall previous versions and track changes of the dataset over time.

In [ ]:
DataChain.from_dataset("embeddings-differences").order_by(C.cos_dist).show(20)

Since we saved our dataset `embeddings-differences` we can now load its content to datachain by the `from_dataset` method. In the above we see the 20 chunks which differ the most between our two embeddings.

We have now solved our scalability issues. When using `DataChain` locally, our computation will still be restricted to a simgle machine but for larger datasets you can use the SaaS version of DataChain available through our DVC Studio which comes with automatic computation cluster management, a graphical user interface and additional ML and data versioning features.

We have also solved our versioning needs and we can track the differences between embeddings over time and use that to choose the best embedding for our use-case.

### Adding more context - merging datasets

In our example bucket, we have not only the `pdf` files themselves but also additinal metadata stored as JSON files. We will now see how we can use Datachain to add the information about authors and the paper title to our `embeddings-differences` dataset which can help us with our evaluation.

In [23]:
import json
meta = json.load(open("sample.json"))
meta["authors"]

[{'given_name': 'Alan', 'family_name': 'Murray', 'institution': None},
 {'given_name': 'Anthony', 'family_name': 'Smith', 'institution': None},
 {'given_name': 'Zoe', 'family_name': 'Butler', 'institution': None}]

As we can see, the metadata contains information about the authors of the paper as a list of dictionaries with each author's name ane institution. Some values can be also be empty. Just as above we create a `DataModel` class to specify the outputs, keeping the name of the file as a key which we will use to join with the previous dataset.

Then we create a function to parse all this information and create a new dataset. We will now only create a single row per original dataset, so we use `return` instead of yield (and there is no need for the `Iterator` class)

In [7]:
# Define the output as a Feature class
class Metadata(DataModel):
    key: str
    title: str
    authors: List[Dict[str, Optional[str]]]


# Use signatures to define input/output types (these can be Feature or regular Python types)
def extract_metadata(file: File) -> Metadata:
    import json
    # Ingest the file
    metadata = json.loads(file.get_value())

    record = dict()
    record["filename"] = file.name.removesuffix("Metadata.json")+"Paper.pdf"
    record["title"] = metadata["title"]
    record["authors"] = metadata["authors"]

    return Metadata(**record)

We now apply the `extract_metadata` function as we did with `pdf_chunks` above, except that we use the `.map` method of DataChain which is employed when there is a 1:1 correspondence between the number of rows of the orignal and the new dataset.

In [8]:
dc_meta = (
    DataChain.from_storage("gs://datachain-demo/neurips")
    .filter(C.name.glob("*Metadata.json"))
    )

(
    dc_meta.
    map(document=extract_metadata)
    .select(
        "document.key",
        "document.title",
        "document.authors",
    )
    .show(3)
)

Processed: 738 rows [00:00, 13196.05 rows/s]
Download: 4.01MB [01:40, 42.0kB/s]
Processed: 184 rows [01:40,  1.84 rows/s]


,document.key,document.title,document.authors
0,02e74f10e0327ad868d138f2b4fdd6f0,Bit-Serial Neural Networks,"[{'given_name': 'Alan', 'family_name': 'Murray..."
1,03afdbd66e7929b125f8597834fa83a4,Connectivity Versus Entropy,"[{'given_name': 'Yaser', 'family_name': 'Abu-M..."
2,072b030ba126b2f4b2374f342be9ed44,The Hopfield Model with Multi-Level Neurons,"[{'given_name': 'Michael', 'family_name': 'Fle..."


[limited by 3 objects]


Finally we will merge the two datasets using the DataChain `.merge` method.

In [ ]:
DataChain.from_dataset("embeddings-differences").merge(dc_meta, on="key").show(20)